In [1]:
from elasticsearch import Elasticsearch
import json
from sentence_transformers import SentenceTransformer

## Step 1: Prepare the documents

In [2]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course in docs_raw:
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

## Step 2: Create Embeddings using Pretrained Models

In [4]:
model = SentenceTransformer("all-mpnet-base-v2")


In [5]:
embeddings = model.encode([doc['text'] for doc in documents])

In [6]:
operations = []
for doc, emb in zip(documents, embeddings):
    doc['text_vector'] = emb.tolist()
    operations.append(doc)

## Step 3: Setup ElasticSearch Connection

In [8]:
es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '4dce495e66a9', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sVTLzVwETtCV6OU0RcV6WQ', 'version': {'number': '8.17.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a091390de485bd4b127884f7e565c0cad59b10d2', 'build_date': '2025-02-28T10:07:26.089129809Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Step 4: Create Mappings and Index

In [14]:
help(es_client.indices.delete)

Help on method delete in module elasticsearch._sync.client.indices:

delete(*, index: Union[str, Sequence[str]], allow_no_indices: Optional[bool] = None, error_trace: Optional[bool] = None, expand_wildcards: Union[Sequence[Union[str, Literal['all', 'closed', 'hidden', 'none', 'open']]], str, Literal['all', 'closed', 'hidden', 'none', 'open'], NoneType] = None, filter_path: Union[str, Sequence[str], NoneType] = None, human: Optional[bool] = None, ignore_unavailable: Optional[bool] = None, master_timeout: Union[str, Literal[-1], Literal[0], NoneType] = None, pretty: Optional[bool] = None, timeout: Union[str, Literal[-1], Literal[0], NoneType] = None) -> elastic_transport.ObjectApiResponse[typing.Any] method of elasticsearch._sync.client.indices.IndicesClient instance
    .. raw:: html

      <p>Delete indices.
      Deleting an index deletes its documents, shards, and metadata.
      It does not delete related Kibana components, such as data views, visualizations, or dashboards.</p>
    

In [16]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text"
            },
            "section": {
                "type": "text"
            },
            "question": {
                "type": "text"
            },
            "course": {
                "type": "keyword"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
       }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents to the index

In [17]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)
    

## Step 6: Create end user query

In [25]:
search_term = "Windows or mac?"
vector_search_term = model.encode(search_term)

## Step 7: Perform Semantic Search

In [29]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [35]:
response = es_client.search(index=index_name, knn=query, source=["course", "section", "question", "text"])
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'xPeJn5UBOb4GhVsdJMlM',
  '_score': 0.71530056,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': '1_eJn5UBOb4GhVsdMsyR',
  '_score': 0.6143398,
  '_source': {'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at one shop:\nYou can use anaconda which has all built in services like pycharm, jupyter\nAdded by Khaja Za